In [1]:
import findspark
findspark.init('/opt/spark')

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('spam_detection').getOrCreate()

22/05/26 14:56:40 WARN Utils: Your hostname, ls-Aspire-A515-51G resolves to a loopback address: 127.0.1.1; using 192.168.15.8 instead (on interface wlp3s0)
22/05/26 14:56:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/26 14:56:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/26 14:56:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [9]:
data = spark.read.csv('smsspamcollection/SMSSpamCollection', inferSchema=True,
                     sep='\t')

In [10]:
data.columns

['_c0', '_c1']

In [11]:
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

In [12]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [13]:
from pyspark.sql.functions import length

In [14]:
data = data.withColumn('length', length(data['text']))

In [15]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [16]:
data.groupBy('class').mean().show() # Feature Engineering

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [17]:
from pyspark.ml.feature import (Tokenizer, 
                               StopWordsRemover,
                                CountVectorizer, 
                               IDF, 
                               StringIndexer)

In [18]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')

In [19]:
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')

In [20]:
count_vec = CountVectorizer(inputCol='stop_token', outputCol='c_vec')

In [21]:
idf = IDF(inputCol='c_vec', outputCol='tf_idf')

In [22]:
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')

In [23]:
from pyspark.ml.feature import VectorAssembler

In [24]:
clean_up = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')

In [25]:
from pyspark.ml.classification import NaiveBayes

In [26]:
nb = NaiveBayes()

In [27]:
from pyspark.ml import Pipeline

In [28]:
data_prep_pipe = Pipeline(stages=[
    ham_spam_to_numeric, 
    tokenizer,
    stop_remove, 
    count_vec, 
    idf,
    clean_up
])

In [29]:
cleaner = data_prep_pipe.fit(data)

In [30]:
clean_data = cleaner.transform(data)

In [31]:
clean_data = clean_data.select('label', 'features')

In [32]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [33]:
train_data, test_data = clean_data.randomSplit([0.7, 0.3])

In [34]:
spam_detector = nb.fit(train_data)

22/05/26 15:09:47 WARN DAGScheduler: Broadcasting large task binary with size 1142.7 KiB
22/05/26 15:09:48 WARN DAGScheduler: Broadcasting large task binary with size 1126.3 KiB


In [35]:
test_results = spam_detector.transform(test_data)

In [36]:
test_results.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [37]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [38]:
acc_eval = MulticlassClassificationEvaluator()

In [39]:
acc = acc_eval.evaluate(test_results)

22/05/26 15:10:52 WARN DAGScheduler: Broadcasting large task binary with size 1365.2 KiB


In [40]:
acc # ACC of NB Model

0.9253237548275942